In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from osrsbox import items_api

from src import fetch_data
from src import data_processing

https://github.com/osrsbox/osrsbox-db?tab=readme-ov-file#item-properties

- id
- name
- last_updated
- cost

In [3]:
df = fetch_data.create_price_df(13024)
df

,timestamp,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume,id
0,1733788800,131759,130153,41,21,13024
1,1733810400,131537,129759,44,11,13024
2,1733853600,131672,129290,112,17,13024
3,1733875200,131289,127825,115,33,13024
4,1733896800,131416,128871,35,16,13024
...,...,...,...,...,...,...
360,1741586400,132076,130348,42,17,13024
361,1741608000,131458,128820,131,21,13024
362,1741629600,132396,126788,117,116,13024
363,1741651200,131246,128334,159,39,13024


In [4]:
df['VWAP'] = data_processing.compute_VWAP(df['avgLowPrice'],df['avgHighPrice'],df['lowPriceVolume'],df['highPriceVolume'])

In [6]:
df

,timestamp,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume,id,VWAP
0,1733788800,131759,130153,41,21,13024,131215.032258
1,1733810400,131537,129759,44,11,13024,131181.400000
2,1733853600,131672,129290,112,17,13024,131358.093023
3,1733875200,131289,127825,115,33,13024,130516.621622
4,1733896800,131416,128871,35,16,13024,130617.568627
...,...,...,...,...,...,...,...
360,1741586400,132076,130348,42,17,13024,131578.101695
361,1741608000,131458,128820,131,21,13024,131093.539474
362,1741629600,132396,126788,117,116,13024,129604.034335
363,1741651200,131246,128334,159,39,13024,130672.424242


In [11]:
pd.to_datetime(df['timestamp'],unit='s')

0     2024-12-10 00:00:00
1     2024-12-10 06:00:00
2     2024-12-10 18:00:00
3     2024-12-11 00:00:00
4     2024-12-11 06:00:00
              ...        
360   2025-03-10 06:00:00
361   2025-03-10 12:00:00
362   2025-03-10 18:00:00
363   2025-03-11 00:00:00
364   2025-03-11 06:00:00
Name: timestamp, Length: 365, dtype: datetime64[ns]

In [9]:
temp = pd.Series([1,2,3,4,5,6])

print(temp/temp.shift(1))
print(temp.shift(-1)/temp)

0         NaN
1    2.000000
2    1.500000
3    1.333333
4    1.250000
5    1.200000
dtype: float64
0    2.000000
1    1.500000
2    1.333333
3    1.250000
4    1.200000
5         NaN
dtype: float64


In [5]:
fetch_data.update_price_file_csv('6h','test_files/test.csv')

test_files/test.csv has been updated


In [5]:
df = pd.read_csv('test_files/test.csv',index_col = 0)

In [7]:
df.sort_values(by=['id','timestamp'])

,timestamp,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume,id
9125,1733637600,173.0,168.0,3925548,1068140,2
9126,1733659200,175.0,171.0,5372467,1351027,2
9127,1733680800,176.0,172.0,5113870,1996143,2
9128,1733702400,179.0,175.0,3796936,1350962,2
9129,1733724000,178.0,173.0,2749820,891517,2
...,...,...,...,...,...,...
7298,1741500000,131380.0,127655.0,91,42,13024
7299,1741521600,131311.0,126941.0,92,107,13024
9911,1741543200,131731.0,127714.0,138,41,13024
9912,1741564800,132047.0,126467.0,138,40,13024
